In [1]:
import numpy as np
import matplotlib.pyplot as plt

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import speedy_physics as radiation_jl

import sys
sys.path.append("models/speedy_physics.f77")
import longwave_radiation_down, longwave_radiation_up

## Compare funcs between fortran and julia

### Downwelling longwave radiation

In [2]:
# Shared
ngp = 4608
nlev = 8
sbc  = 5.67e-8
emisfc = 0.98
epslw  =  0.05
wvi = np.full((nlev, 2), 1.)
tau2 = np.full((ngp, nlev, 4), 0.5)
ta = np.full((ngp, nlev), 300.)
ts = np.full(ngp, 320.)

# Init F77 common data
longwave_radiation_down.phycon.sbc  = sbc
longwave_radiation_down.radcon.emisfc = emisfc
longwave_radiation_down.radcon.epslw  =  epslw
longwave_radiation_down.fsiglt.wvi = wvi
longwave_radiation_down.radfld.tau2 = tau2

# Inputs F77 only
imode = -1 # Downwelling only

longwave_radiation_down_f_data = imode, ta, ts

# Inputs Julia
fband = radiation_jl.radset()
longwave_radiation_down_jl_data = ta, fband, epslw, emisfc, sbc, wvi, tau2, ngp

fsfcd_f, _, _, _, dfabs_f = longwave_radiation_down.radlw(*longwave_radiation_down_f_data)
fsfcd_jl, dfabs_jl, flux_jl, st4a_jl = radiation_jl.radlw_down(*longwave_radiation_down_jl_data)

np.testing.assert_allclose(fsfcd_f, fsfcd_jl, rtol=1e-06, atol=1e-6)
np.testing.assert_allclose(dfabs_f, dfabs_jl, rtol=1e-06, atol=1e-6)

### Upwelling longwave radiation

In [3]:
# Shared
ngp = 4608
nlev = 8
sbc  = 5.67e-8
emisfc = 0.98
epslw  =  0.05
wvi = np.full((nlev, 2), 0.5)
tau2 = np.full((ngp, nlev, 4), 0.5)
stratc = np.full((ngp, 2), 0.5)
dsig = np.full(nlev, 0.5)
ta = np.full((ngp, nlev), 300.)
ts = np.full(ngp, 320.)
fsfcu = radiation_jl.compute_bbe(ts, ngp)

# Init F77 common data
longwave_radiation_up.phycon.sbc  = sbc
longwave_radiation_up.radcon.emisfc = emisfc
longwave_radiation_up.radcon.epslw  =  epslw
longwave_radiation_up.radfld.stratc = stratc
longwave_radiation_up.fsiglt.dsig = dsig
longwave_radiation_up.radfld.tau2 = tau2
longwave_radiation_up.radfld.st4a = st4a_jl
longwave_radiation_up.radfld.flux = flux_jl

# Inputs F77
imode = 1 # Upwelling only
longwave_radiation_up_f_data = imode, ta, ts, fsfcd_f, fsfcu, dfabs_f

# Inputs Julia
fband = radiation_jl.radset()
longwave_radiation_up_jl_data = ta, ts, fsfcd_jl, fsfcu, dfabs_jl, fband, flux_jl, st4a_jl, emisfc, epslw, dsig, tau2, stratc, ngp

fsfc_f, ftop_f = longwave_radiation_up.radlw(*longwave_radiation_up_f_data)
fsfc_jl, ftop_jl = radiation_jl.radlw_up(*longwave_radiation_up_jl_data)

np.testing.assert_allclose(fsfc_f, fsfc_jl, rtol=1e-06, atol=1e-6)
np.testing.assert_allclose(ftop_f, ftop_jl, rtol=1e-06, atol=1e-6)

## Generate I/Os for Julia unit tests

In [4]:
# Checking the last band will suffice.
fband = radiation_jl.radset()
print(fband[-1, :])

[0.19498351 0.12541235 0.33106664 0.2985375 ]


In [5]:
ngp = 1
nlev = 8

ta = np.full((ngp, nlev), 300.)
fband = radiation_jl.radset()
epslw  =  0.05
emisfc = 0.98
sbc  = 5.67e-8
wvi = np.full((nlev, 2), 0.5)
tau2 = np.full((ngp, nlev, 4), 0.5)

fsfcd, dfabs, flux, st4a = radiation_jl.radlw_down(ta, fband, epslw, emisfc, sbc, wvi, tau2, ngp)
print(fsfcd)
print(dfabs)

[447.29436216]
[[ -71.86727228 -182.21961386  -91.10980693  -45.55490346  -22.77745173
   -11.38872587   -5.69436293  -25.35141147]]


In [6]:
ngp = 1
ts = np.full(ngp, 320.)
fsfcu = radiation_jl.compute_bbe(ts, ngp)
print(fsfcu)

[582.65174016]


In [7]:
ngp = 1
nlev = 8

ta = np.full((ngp, nlev), 300.)
ts = np.full(ngp, 320.)
fsfcu = radiation_jl.compute_bbe(ts, ngp)
fband = radiation_jl.radset()
emisfc = 0.98
epslw  =  0.05
sbc  = 5.67e-8
dsig = np.full(nlev, 0.5)
tau2 = np.full((ngp, nlev, 4), 0.5)
stratc = np.full((ngp, 2), 0.5)
wvi = np.full((nlev, 2), 0.5)
fsfcd, dfabs, flux, st4a = radiation_jl.radlw_down(ta, fband, epslw, emisfc, sbc, wvi, tau2, ngp)

fsfc, ftop = radiation_jl.radlw_up(ta, ts, fsfcd, fsfcu, dfabs, fband, flux, st4a, emisfc, epslw, dsig, tau2, stratc, ngp)
print(fsfc)
print(ftop)

[135.357378]
[667.24601389]
